In [1]:
!git clone https://github.com/dna-witch/KGPL-PyTorch/ -b th-dev

Cloning into 'KGPL-PyTorch'...
remote: Enumerating objects: 459, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 459 (delta 35), reused 17 (delta 17), pack-reused 400 (from 1)
Receiving objects: 100% (459/459), 15.46 MiB | 19.21 MiB/s, done.
Resolving deltas: 100% (195/195), done.


In [2]:
!pip install -r KGPL-PyTorch/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [ ]:
!cd KGPL-PyTorch &&\
python preprocess/preprocess.py -d "music" &&\
python preprocess/make_path_list.py lp_depth=6 dataset=music kg_path=data/music/kg_final.npy rating_path=data/music/ratings_final.npy num_neighbor_samples=32

reading item index to entity id file: data/music/item_index2entity_id.txt ...
reading rating file ...
converting rating file ...
number of users: 1872
number of items: 3846
converting kg file ...
number of entities (containing items): 9366
number of relations: 60
done
adj_entity_path: data/music/adj_entity_6_32.npy
adj_relation_path: data/music/adj_relation_6_32.npy
data_path: data/music/fold1.pkl
dataset: music
kg_path: data/music/kg_final.npy
lp_depth: 6
num_neighbor_samples: 32
pathlist_path: data/music/path_list_6_32.pkl
rating_path: data/music/ratings_final.npy
reachable_items_path: data/music/reachable_items.pkl

[Parallel(n_jobs=32)]: Using backend MultiprocessingBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Batch computation too fast (0.016800403594970703s.) Setting batch_size=2.
[Parallel(n_jobs=32)]: Done   8 tasks      | elapsed:    0.2s
[Parallel(n_jobs=32)]: Done  21 tasks      | elapsed:    0.5s
[Parallel(n_jobs=32)]: Done  34 tasks      | elapsed:    1.0s
[P

In [ ]:
import sys
sys.path.append('KGPL-PyTorch/')

In [ ]:
# from code
from model import *
from utils.data_classes import *
from utils.functions import *
from utils.eval import *

# libraries
import torch
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
music = KGPLExperiment(
    'music',
    {
      'plabel_lp_depth': 6,
      'plabel_par': 16,
      'plabel_chunk_size': 250,
      'plabel_neg_pn': 0.1,
      'plabel_pl_pn': 1e-3,
      'dropout_rate': 0.5,
      'emb_dim': 64,
      'n_iter': 1,
      'plabel': {},
      'optimize': {'iter_per_epoch': 100, 'lr': 3e-3, 'batch_size': 3333},
      'log': {'show_loss': True},
      'evaluate': {'user_num_topk': 1000},
      'model': {
          'n_iter': 1,
          'neighbor_sample_size': 32,
          'num_neighbor_samples': 32,
          'dropout_rate': 0.5}
    })

In [ ]:
music.train_val_test_split()

In [ ]:
music.create_dataloaders()

In [ ]:
model = KGPLCOT(
        music.cfg,
        int(music.ratings[:, 0].max().item()) + 1,
        int(music.ratings[:, 1].max().item()) + 1,
        music.n_entity,
        music.n_relation,
        music.adj_entity,
        music.adj_relation,
        music.path_list_dict,
        music.train_loader,
        music.val_loader,
        'cuda'
    )

In [ ]:
def run_cotrain(model, train_loader, val_loader, test_loader, cfg, device, num_epochs=100):
    '''
    Run the CoTrain algorithm with the given model, data loaders, configuration, and device, using the "music" experiment.
    Optimizers are defined here with epsilon = 1e-7 to match tensorflow default.

    Inputs:
    - model: The CoTrain model to be trained.
    - train_loader: The data loader for the training set.
    - val_loader: The data loader for the validation set.
    - test_loader: The data loader for the test set.
    - cfg: The configuration dictionary.
    - device: PyTorch device
    - num_epochs: Training epochs

    Returns:
    - pd.DataFrame: A DataFrame containing the training loss and validation metrics for each epoch.
    '''
    optimizer_f = torch.optim.Adam(
        model.model_f.parameters(),
        lr=cfg['optimize']['lr'],
        eps=1e-7,
      )
    optimizer_g = torch.optim.Adam(
        model.model_g.parameters(),
        lr=cfg['optimize']['lr'],
        eps=1e-7,
      )
    iter_per_epoch = cfg['optimize']['iter_per_epoch']

    dataframe_rows = []

    for epoch in range(1, num_epochs + 1):
        print(f"\nEpoch {epoch}")
        model.train()
        total_loss_f, total_loss_g = 0.0, 0.0

        train_iter = iter(train_loader)  # re-initialize iterator every epoch

        for i in tqdm(range(iter_per_epoch)):
            try:
                batch = next(train_iter)
            except StopIteration:
                train_iter = iter(train_loader)
                batch = next(train_iter)

            users, pos, neg, pseudo = [b.to(device) for b in batch]
            batch_data = torch.stack([users, pos, neg, pseudo], dim=1)

            losses = model.train_step(batch_data, optimizer_f, optimizer_g)
            total_loss_f += losses['loss_f']
            total_loss_g += losses['loss_g']

        avg_loss_f = total_loss_f / iter_per_epoch
        avg_loss_g = total_loss_g / iter_per_epoch
        print(f"Train Loss - f: {avg_loss_f:.4f}, g: {avg_loss_g:.4f}")

        print("Evaluating model_f:")

        row = pd.DataFrame(
            {"Epoch": epoch,
             "Train Loss F": avg_loss_f,
             "Train Loss G": avg_loss_g
             }, index=[0])

        model.eval()

        print("Validation:")
        val_row = run_topk_eval(
          model=model,
          cfg = cfg,
          train_data=music.train_dataset.ratings.numpy(),
          eval_data=music.val_dataset.ratings.numpy(),
          test_data=music.test_dataset.ratings.numpy(),
          n_items=music.n_item,
          device=device,
          test_mode=False  # or True for test set
        ).add_prefix("Validation ")

        row = row.join(val_row)

        dataframe_rows.append(row)
    return pd.concat(dataframe_rows, ignore_index=True)

In [ ]:
results = run_cotrain(
    model,
    music.train_loader,
    music.val_loader,
    music.test_loader,
    music.cfg,
    'cuda',
    num_epochs=40
)

In [ ]:
sns.lineplot(results, x='Epoch', y='Validation Recall@5', label='Recall@5')
sns.lineplot(results, x='Epoch', y='Validation Recall@10', label='Recall@10')
sns.lineplot(results, x='Epoch', y='Validation Recall@20', label='Recall@20')
plt.title('Validation Recall of Model F')

In [ ]:
sns.lineplot(results, x='Epoch', y='Train Loss F')
plt.title('Train Loss of Model F')

In [ ]:
# Get unique values and their counts
unique_users, counts = torch.unique(music.train_dataset.ratings[:,0], return_counts=True)

In [ ]:
print('All Users:', len(unique_users), '\n')

cold_starts = pd.Series(range(1,11), name='Cold Start Values')

cold_start_rows = []

for i in cold_starts:
  # Select values with count <= i
  cold_start_rows.append(cold_start_eval(model, music, i, 'cuda'))
  print('\n')

In [ ]:
# Cold Start Test Set
cold_starts.to_frame().join(pd.concat(cold_start_rows, ignore_index=True))

In [ ]:
# Overall Test Set Evaluation
run_topk_eval(
          model=model,
          cfg = music.cfg,
          train_data=music.train_dataset.ratings.numpy(),
          eval_data=music.val_dataset.ratings.numpy(),
          test_data=music.test_dataset.ratings.numpy(),
          n_items=music.n_item,
          device='cuda',
          test_mode=True  # or True for test set
        )